# Advent of Code 2018

### Day 1

In [199]:
@time let input = parse.(Int,eachline("input1"))
    freq = 0
    history = Set(0)
    found = false
    while !found
        for x = input
            freq += x
            if freq in history
                found = true
                break
            end
            push!(history, freq)
        end
    end
    sum(input), freq
end

  0.009071 seconds (2.08 k allocations: 3.073 MiB)


(416, 56752)

### Day 2

In [196]:
function almost_the_same(a,b)
    diffFlag = false
    for (c,d) = zip(a,b)
        if c != d
            if diffFlag
                return false
            end
            diffFlag = true
        end
    end
    return diffFlag # Technically if they are equal they aren't "almost" the same
end
@time let input = readlines("input2")
    twos, threes = sum(input) do line # O(nmlogm)
        chars = sort(line |> collect) # O(mlogm) sorted so now all elements are one after another
        counts = [0,0]
        i = 1
        while i <= length(chars) # O(m)
            j = findnext(!isequal(chars[i]),chars,i)
            j == nothing && (j = length(chars)+1)
            if 2 <= j-i <= 3
                counts[j-i-1] += 1
            end
            i = j
        end
        counts
    end
    chksum = twos * threes
    i,j = first(
        (i,j)
        for i = 1:length(input)
        for j = i+1:length(input)
        if almost_the_same(input[i],input[j]))
    chksum, first.(filter(x->(==)(x...),zip(input[i],input[j]) |> collect)) |> join
end

  0.210266 seconds (254.25 k allocations: 12.193 MiB)


(15264, "tiwcdpbseqhxryfmgkvjujvza")

### Day 3

In [173]:
@time begin
    rects = map(eachline("input3")) do line
        pos, dims = split(line)[3:4]
        pos = parse.(Int,split(chop(pos),','))
        dims = pos .+ parse.(Int,split(dims,'x'))
        (broadcast((p,d) -> 1+p:d, pos, dims)) |> Tuple |> CartesianIndices
    end
    grid = zeros(Int, 1000,1000)
    for rect in rects
        grid[rect] .+= 1
    end
    
    i = findfirst(rects) do rect
        all(isequal(1),view(grid,rect))
    end
    
    count(x->x>1,grid), i
end

  0.285721 seconds (447.25 k allocations: 32.650 MiB, 3.20% gc time)


(118539, 1270)

### Day 4

In [82]:
using SplitApplyCombine, Dates
@time let logs = map(eachline("input4")) do line
        m = match(r"^\[(.*?)\] (?:Guard #(\d+) )?(.*?)$", line)
        dt = DateTime(first(m.captures),"y-m-d H:M")
        dt => something.(m.captures[2:3],"") |> Vector{String}
    end
    sort!(logs,by=first)
    
    for (i,log) in enumerate(logs)
        if isempty(log.second[1])
            log.second[1] = logs[i-1].second[1]
        end
    end
    
    filter!(x->x |> last |> last |> !isequal("begins shift"),logs)
    
    days = [first(last(first(v))) => map(x->x[1]=>x[2][2]=="wakes up",v) for (k,v) in group(dayofyear ∘ first, logs)]
    
    days = map(days) do (id,day)
        arr = zeros(Bool,60)
        
        for (a,b) in Iterators.partition(day,2)
            arr[1+minute(a|>first) : minute(b|>first)] .= true
        end
        id=>arr
    end
    
    guards = Dict(k=>hcat(v...) for (k,v) in group(first,last,days))
    
    _,id = findmax(Dict(k=>sum(v) for (k,v) in guards))
    
    _,min = findmax(sum(guards[id],dims=2))
    
    ans1 = parse(Int,id) * (min[1]-1)
    
    bestTimes = Dict(k=>findmax(sum(v, dims=2)) for (k,v) in guards)
    
    _,id = findmax(Dict(k=>first(v) for (k,v) in bestTimes))
    
    @show last(bestTimes[id])
    ans2 = parse(Int,id) * (last(bestTimes[id])[1]-1)
    
    ans1,ans2
end

last(bestTimes[id]) = CartesianIndex(37, 1)
  0.645827 seconds (925.75 k allocations: 45.254 MiB, 2.94% gc time)


(65489, 3852)

### Day 5

In [83]:
iscomplement(a,b) = a!=b && lowercase(a)==lowercase(b)
function reduce_polymer(polymer)
    arr = Char[]
    for c in polymer
        if isempty(arr) || !iscomplement(last(arr), c)
            push!(arr,c)
        else
            pop!(arr)
        end
    end
    arr
end
@time let code = readline("input5") |> collect
    l = length(code |> reduce_polymer)
    minl = minimum( length(filter(c->lowercase(c)!=unit, code) |> reduce_polymer) for unit in 'a':'z')
    l,minl
end

  0.162378 seconds (91.49 k allocations: 20.911 MiB, 4.40% gc time)


(10638, 4944)

### Day 6

In [3]:
const symbols = vcat('a':'z','A':'Z')
const offsets = CartesianIndex.([(1,0),(-1,0),(0,1),(0,-1)])
neighbours(p,sz) = filter(x->x in Carte,p.+offsets)

52-element Array{Char,1}:
 'a'
 'b'
 'c'
 'd'
 'e'
 'f'
 'g'
 'h'
 'i'
 'j'
 'k'
 'l'
 'm'
 ⋮  
 'O'
 'P'
 'Q'
 'R'
 'S'
 'T'
 'U'
 'V'
 'W'
 'X'
 'Y'
 'Z'

In [23]:
@time let points = map(line->parse.(Int,line) |> Tuple |> CartesianIndex,split.(eachline("input6"),", "))
    offset = minimum(points) - CartesianIndex(1,1)
    map!(x->x-offset,points, points)
    
    sz = maximum(points)
    
    grid = fill(' ', Tuple(sz))
    for (i,p) in enumerate(points)
        grid[p] = symbols[i]
    end
    
    @time for i in keys(grid)
        if grid[i] == ' '
            idx,dist,tie = nothing,Inf,false
            for (j,p) in enumerate(points)
                dst = sum(abs.(Tuple(i-p)))
                if dst < dist
                    idx,dist,tie = j,dst,false
                elseif dst == dist
                    tie = true
                end
            end
            grid[i] = tie ? '.' : symbols[idx]
        end
    end
    
    infinitesymbols = Set(vcat(grid[1,:], grid[end,:], grid[:,1], grid[:,end]))
    finitesymbols = filter(x->!(x in infinitesymbols),symbols)
    maximum(count(isequal(symbol), grid) for symbol in finitesymbols)
end

 32.180404 seconds (67.32 M allocations: 2.071 GiB, 1.06% gc time)
 32.644930 seconds (67.67 M allocations: 2.088 GiB, 1.05% gc time)


5941

In [28]:
@time let points = map(line->parse.(Int,line) |> Tuple |> CartesianIndex,split.(eachline("input6"),", "))
    offset = minimum(points) - CartesianIndex(1,1)
    map!(x->x-offset,points, points)
    
    sz = maximum(points)
    
    count(CartesianIndices(sz)) do i
        sum(p->sum(abs.(Tuple(i-p))),points) < 10000
    end
end

  0.249727 seconds (231.95 k allocations: 11.444 MiB, 5.60% gc time)


40244